In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import nn
import os
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#tpr = torch.hub.load('yangsenius/TransPose:main', 'tpr_a4_256x192', pretrained=True)
tph = torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True, device=device)


#print(tph)
DATASET_PATH = './dataset/'
positions = os.listdir(DATASET_PATH)
#print(os.listdir(DATASET_PATH+entries[0]))
images = list()



class YogaPoseDataset(Dataset):

    def __init__(self, dataset_path):
        self.data_path = dataset_path
        # call to init the data

        self._init_data()

    def _init_data(self):
        positions = os.listdir(self.data_path)
        images = list()
        for idx, position in enumerate(positions):
            for file in os.listdir(DATASET_PATH + position):
                with Image.open(DATASET_PATH + position + '/' + file, "r") as f:
                    # f.show()
                    data = np.asarray(f)
                    images.append((idx, data))
                    # print(data)
        self.images = np.array(images)
        np.random.shuffle(self.images)

    def __len__(self):
        # returns the number of samples in our dataset
        return len(self.images)

    def __getitem__(self, idx):
        # returns the idx-th sample
        return self.images[idx]

Using cache found in /home/michele/.cache/torch/hub/yangsenius_TransPose_main
/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_h.py:514: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature ** (2 * (dim_t // 2) / one_direction_feats)


>>Load pretrained weights from url: https://github.com/yangsenius/TransPose/releases/download/Hub/tp_h_48_256x192_enc4_d96_h192_mh1.pth
Successfully loaded model  (on cpu) with pretrained weights!


In [2]:
dataset = YogaPoseDataset(DATASET_PATH)
split_position = int((len(dataset)/10)*7)
trainset = dataset[:split_position]
testset = dataset[split_position:]
print(trainset)

[[101 array([[[221, 227, 223],
              [220, 226, 222],
              [220, 226, 222],
              ...,
              [242, 252, 253],
              [242, 252, 253],
              [242, 252, 253]],

             [[220, 226, 222],
              [221, 227, 223],
              [221, 227, 223],
              ...,
              [243, 253, 254],
              [243, 253, 254],
              [243, 253, 254]],

             [[219, 225, 221],
              [221, 227, 223],
              [221, 227, 223],
              ...,
              [243, 253, 254],
              [244, 254, 255],
              [243, 253, 254]],

             ...,

             [[ 35,  51, 100],
              [ 33,  54,  99],
              [ 32,  53,  98],
              ...,
              [ 57,  99, 157],
              [ 57,  99, 157],
              [ 56,  98, 156]],

             [[ 27,  54,  99],
              [ 23,  56,  99],
              [ 23,  56,  99],
              ...,
              [ 55,  98, 141],
          

/tmp/ipykernel_6809/4155742358.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.images = np.array(images)


In [ ]:
class PoseClassifier(nn.Module):
    def __init__(self, n_class, fine_tune, pretrained=True):
        super(PoseClassifier, self).__init__()

        self.tph = torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True, device=device)
        self.fc1 = nn.Linear(1000,1000).to(device)
        self.fc2 = nn.Linear(1000,n_class).to(device)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = x.to(device)
        out = self.tph(x)
        out = self.fc1(out)
        #m = nn.BatchNorm1d(1000,device=device)
        #out = m(out)
        out = self.relu(out)
        out = self.fc2(out)
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return out


In [ ]:
X = np.array(dataset[:,1],dtype=float)
#X = X.astype(float)
out = tph(torch.tensor(X))